# HoGent GPT Workshop
This notebook will elaborate on how to use OpenAI's GPT-3 model and explain various use cases on how to leverage the power of a Large Language Model (LLM). We will use Python and the REST-services provided by OpenAI to expore our use case.

In [3]:
# Import all the needed libraries
import configparser, pandas as pd, os, ssl, sqlalchemy, openai
from gpt_index import SQLDatabase, LLMPredictor
from langchain import OpenAI, SQLDatabaseChain
from sqlalchemy import create_engine

# Set the API key and load the configuration file
config = configparser.ConfigParser()
config.read('config.ini')
openai.api_key = config['openai']['api_key']
os.environ['OPENAI_API_KEY'] = config['openai']['api_key']

In [3]:
# Load an sqlalchmy engine  to connect to the database
# Create the connection url
connection_string = 'postgresql+pg8000://' + config['postgres']['user'] + ':' + config['postgres']['password'] + '@' + config['postgres']['host'] + '/' + config['postgres']['database']
engine = create_engine(connection_string)  


## Write SQL Queries using natural language
First, we will use GPT-3 to generate SQL statements and run them againts a database to provide answers to our questions using natural language. Implementing a solution like this can help business user without technical knowledge to run queries against there internall database.

### Example Table **belgian_names**: First names of the total population by municipality
This database is loaded with Statbel data found here: https://statbel.fgov.be/en/open-data/first-names-total-population-municipality-12

![Alt text](./img/names_table.png)

In [4]:
# Create a connection
connection = engine.connect()
# Load the table we want to query
postgres_database = SQLDatabase(connection, include_tables=["belgian_names"])
# Show the table information
postgres_database.table_info

"Table 'belgian_names' has columns: nis_code (VARCHAR(50)), city_name (VARCHAR(50)), firstname (VARCHAR(50)), frequency (INTEGER), sex (VARCHAR)."

In [5]:
# Let's initiate our model 
llm = OpenAI(temperature=0)
db_chain = SQLDatabaseChain(llm=llm, database=postgres_database, verbose=True)

In [21]:
db_chain.run('Is there a difference in average lenght between male an female names ?')



> Entering new SQLDatabaseChain chain...
Is there a difference in average lenght between male an female names ? 
SQLQuery: SELECT AVG(LENGTH(firstname)) AS avg_length, sex FROM belgian_names GROUP BY sex;
SQLResult: [(Decimal('6.2237759349429328'), 'FEMALE'), (Decimal('5.8008796787083095'), 'MALE')]
Answer: Yes, there is a difference in average length between male and female names, with female names being on average 0.42 characters longer than male names.
> Finished chain.


' Yes, there is a difference in average length between male and female names, with female names being on average 0.42 characters longer than male names.'

## Generate descriptions for a list of products

In [4]:
beers = pd.read_csv('./data_files/beers.csv')

In [5]:
# Call the OpenAI completion endpoint 


,Name,Brewery,Type
0,Budweiser,Anheuser-Busch,Lager
1,Coors Light,Coors Brewing Company,Light Lager
2,Miller Lite,Miller Brewing Company,Light Lager
3,Corona Extra,Grupo Modelo,Pale Lager
4,Heineken,Heineken Brewery,Pale Lager
5,Guinness Draught,Guinness,Stout
6,Samuel Adams Boston Lager,Boston Beer Company,Amber Lager
7,Stella Artois,Stella Artois Brewery,Pilsner
8,Pabst Blue Ribbon,Pabst Brewing Company,Amber Lager
9,Blue Moon Belgian White,Blue Moon Brewing Company,Belgian-style Wheat Ale
